In [ ]:
import os
import sys
import scipy.io
import scipy.misc
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from PIL import Image
import numpy as np
import tensorflow as tf
import pprint
from cost_functions import *
from misc_utils import *
#from public_tests import *
%matplotlib inline

Load VGG19 

In [ ]:
img_size = 300
vgg = tf.keras.applications.VGG19(include_top=False,
                                  input_shape=(img_size, img_size, 3),
                                  weights='imagenet')

vgg.trainable = False

Chose your style and content layers. Uncomment the top to list all layers

In [ ]:
#Chose layers to use when calculating style cost

#for layer in vgg.layers:
#    print(layer.name)

STYLE_LAYERS = [
    ('block2_conv1', 0.1),
    ('block2_conv2', 0.1),
    ('block3_conv1', 0.15),
    ('block3_conv2', 0.2),
    ('block3_conv3', 0.3),
    ('block4_conv1', 0.5),
    ('block4_conv2', 0.5),
    ('block4_conv3', 0.5),
    ('block4_conv4', 0.5),
    ('block5_conv1', 0.2),
    ('block5_conv2', 0.1),
    ('block5_conv3', 0.1),
    ('block5_conv4', 0.1)]

#CONTENT_LAYERS = [('block5_conv4', 2)]
CONTENT_LAYERS = [('block3_conv2', 2),
                ('block4_conv1', 1),
                 ('block4_conv2', 1),
                ('block4_conv3', 1),
                ('block4_conv4', 1)]
INPUT_LAYERS = [(vgg.layers[0].name, 1)]

vgg_style_outputs = get_layer_outputs(vgg, STYLE_LAYERS)
vgg_model_content_outputs = get_layer_outputs(vgg, CONTENT_LAYERS)
inputs_layer_output = get_layer_outputs(vgg, INPUT_LAYERS)


In [ ]:
content_image = np.array(Image.open("images/louvre_small.jpg").resize((img_size, img_size)))
content_image = tf.constant(np.reshape(content_image, ((1,) + content_image.shape)))

print(content_image.shape)
imshow(content_image[0])
plt.show()

In [ ]:
style_image =  np.array(Image.open("images/monet.jpg").resize((img_size, img_size)))
style_image = tf.constant(np.reshape(style_image, ((1,) + style_image.shape)))

print(style_image.shape)
imshow(style_image[0])
plt.show()

In [ ]:
content_image2 = np.array(Image.open("images/louvre_small.jpg").resize((img_size, img_size)))
content_image2 = tf.constant(np.reshape(content_image2, ((1,) + content_image2.shape)))
generated_image = tf.Variable(tf.image.convert_image_dtype(content_image2, tf.float32))
#generated_image = tf.Variable(tf.zeros(tf.shape(generated_image))); generated_image = tf.add(generated_image, 0.5)
noise = tf.random.uniform(tf.shape(generated_image), -0.1, 0.1)
#generated_image = tf.add(generated_image, noise)
generated_image = tf.clip_by_value(generated_image, clip_value_min=0.0, clip_value_max=1.0)

print(generated_image.shape)
imshow(generated_image.numpy()[0])
plt.show()
generated_image = tf.Variable(generated_image)

In [ ]:
# Assign the content image to be the input of the VGG model.  
# Set a_C to be the hidden layer activation from the layer we have selected
preprocessed_content =  tf.Variable(tf.image.convert_image_dtype(content_image, tf.float32))
a_C = vgg_model_content_outputs(preprocessed_content)
# Assign the input of the model to be the "style" image 
preprocessed_style =  tf.Variable(tf.image.convert_image_dtype(style_image, tf.float32))
a_S = vgg_style_outputs(preprocessed_style)

content_target = vgg_style_outputs(content_image)  # Content encoder
style_targets = vgg_style_outputs(style_image)     # Style encoder

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

#@tf.function()
def train_step(generated_image):
    with tf.GradientTape() as tape:
        # In this function you must use the precomputed encoded images a_S and a_C
        
        ### START CODE HERE
        
        # Compute a_G as the vgg_style_outputs for the current generated image
        #(1 line)
        a_G_s = vgg_style_outputs(generated_image)
        a_G_c = vgg_model_content_outputs(generated_image)
        
        # Getting the original image for denoising
        a_I = inputs_layer_output(generated_image)
        
        # Compute the style cost
        #(1 line)
        J_style = compute_style_cost(a_S, a_G_s, STYLE_LAYERS)
        
        #(2 lines)
        # Compute the content cost
        J_content = compute_content_cost(a_C,a_G_c)
        
        
        J_noise = compute_noise_cost(a_I)
        
        # Compute the total cost
        J = total_cost(J_content, J_style, J_noise, alpha = 50, beta = 250, gamma = 0.001)
        
        ### END CODE HERE
        
    grad = tape.gradient(J, generated_image)

    optimizer.apply_gradients([(grad, generated_image)])
    generated_image.assign(clip_0_1(generated_image))
    # For grading purposes
    return J

In [ ]:
# Show the generated image at some epochs
# Uncomment to reset the style transfer process. You will need to compile the train_step function again 
epochs = 2501
for i in range(epochs):
    train_step(generated_image)
    if i % 25 == 0:
        print(f"Epoch {i} ")
    if i % 250 == 0:
        image = tensor_to_image(generated_image)
        imshow(image)
        image.save(f"output/image_{i}.jpg")
        plt.show() 

In [ ]:
# Show the 3 images in a row
fig = plt.figure(figsize=(16, 4))
ax = fig.add_subplot(1, 3, 1)
imshow(content_image[0])
ax.title.set_text('Content image')
ax = fig.add_subplot(1, 3, 2)
imshow(style_image[0])
ax.title.set_text('Style image')
ax = fig.add_subplot(1, 3, 3)
imshow(generated_image[0])
ax.title.set_text('Generated image')
plt.show()